In [6]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os

In [7]:
df1 = pd.read_csv("case_study1.xlsx - case_study1.csv")

In [8]:
df2 = pd.read_csv("case_study2.xlsx - case_study2.csv")

In [9]:
pd.options.display.max_columns = None
df1.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,Total_TL_opened_L12M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,Auto_TL,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,0,0,0.00,0.0,0,0,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,1,0,1.00,0.0,0,0,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,2,0,0.25,0.0,1,1,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.0,1.000,0.000,1,0,1.00,0.0,1,0,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.0,0.333,0.667,0,0,0.00,0.0,0,1,0,0,0,0,0,3,0,2,131,32


In [10]:
df1.shape

(51336, 26)

In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

In [12]:
df1.isnull().sum()

PROSPECTID              0
Total_TL                0
Tot_Closed_TL           0
Tot_Active_TL           0
Total_TL_opened_L6M     0
Tot_TL_closed_L6M       0
pct_tl_open_L6M         0
pct_tl_closed_L6M       0
pct_active_tl           0
pct_closed_tl           0
Total_TL_opened_L12M    0
Tot_TL_closed_L12M      0
pct_tl_open_L12M        0
pct_tl_closed_L12M      0
Tot_Missed_Pmnt         0
Auto_TL                 0
CC_TL                   0
Consumer_TL             0
Gold_TL                 0
Home_TL                 0
PL_TL                   0
Secured_TL              0
Unsecured_TL            0
Other_TL                0
Age_Oldest_TL           0
Age_Newest_TL           0
dtype: int64

#### There are some values in 'Age_Oldest_TL' written in the form of -99999 which is considered as null values. So we have to deal with them

In [13]:
df1.sample(10)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,Total_TL_opened_L12M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,Auto_TL,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
29769,29770,1,1,0,0,1,0.000,1.000,0.000,1.000,0,1,0.000,1.000,0,0,0,1,0,0,0,0,1,0,14,14
1661,1662,1,0,1,0,0,0.000,0.000,1.000,0.000,1,0,1.000,0.000,0,0,0,1,0,0,0,0,1,0,10,10
28210,28211,2,0,2,1,0,0.500,0.000,1.000,0.000,2,0,1.000,0.000,0,0,0,2,0,0,0,0,2,0,7,3
44287,44288,4,3,1,0,0,0.000,0.000,0.250,0.750,0,1,0.000,0.250,1,0,0,0,3,0,0,4,0,1,167,22
14432,14433,3,1,2,0,1,0.000,0.333,0.667,0.333,0,1,0.000,0.333,0,0,0,0,0,0,0,2,1,3,54,23
17201,17202,2,0,2,1,0,0.500,0.000,1.000,0.000,2,0,1.000,0.000,1,0,0,1,0,0,0,1,1,1,12,3
3709,3710,9,1,8,3,1,0.333,0.111,0.889,0.111,3,1,0.333,0.111,0,1,4,2,0,0,1,1,8,1,70,2
25585,25586,2,2,0,0,1,0.000,0.500,0.000,1.000,0,1,0.000,0.500,0,0,0,2,0,0,0,0,2,0,26,15
28346,28347,1,1,0,0,0,0.000,0.000,0.000,1.000,0,0,0.000,0.000,0,0,0,0,1,0,0,1,0,0,54,54
37910,37911,6,3,3,0,2,0.000,0.333,0.500,0.500,1,2,0.167,0.333,0,2,0,3,0,0,0,2,4,1,36,8


In [14]:
df1[df1['Age_Oldest_TL'] == -99999]['Age_Oldest_TL'].count()

40

In [15]:
df1.shape

(51336, 26)

### There are '40' null values in 'Age_Oldest_TL' feature 

In [16]:
# Remove all the null containing in this column
df1 = df1.loc[df1['Age_Oldest_TL'] != -99999]
print(df1.shape)

(51296, 26)


In [17]:
pd.options.display.max_columns = None
df2.head()

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,max_deliq_6mts,max_deliq_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,num_std_6mts,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_6mts,num_dbt_12mts,num_lss,num_lss_6mts,num_lss_12mts,recent_level_of_deliq,tot_enq,CC_enq,CC_enq_L6m,CC_enq_L12m,PL_enq,PL_enq_L6m,PL_enq_L12m,time_since_recent_enq,enq_L12m,enq_L6m,enq_L3m,MARITALSTATUS,EDUCATION,AGE,GENDER,NETMONTHLYINCOME,Time_With_Curr_Empr,pct_of_active_TLs_ever,pct_opened_TLs_L6m_of_L12m,pct_currentBal_all_TL,CC_utilization,CC_Flag,PL_utilization,PL_Flag,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,-99999,-99999,0,0,21,5,11,0,0,0,0,0,0,0,0,0,29,6,0,0,0,6,0,0,566,0,0,0,Married,12TH,48,M,51000,114,0.200,0.0,0.798,-99999.0,0,0.798,1,0.0,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,209,1,0,0,Single,GRADUATE,23,F,19000,50,1.000,0.0,0.370,-99999.0,0,-99999.000,0,0.0,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,25,25,0,0,10,5,10,0,0,0,0,0,0,0,0,0,25,4,0,0,0,0,0,0,587,0,0,0,Married,SSC,40,M,18,191,1.000,0.5,0.585,-99999.0,0,-99999.000,0,0.0,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,-99999,-99999,0,0,5,4,5,0,0,0,0,0,0,0,0,0,0,-99999,-99999,-99999,-99999,-99999,-99999,-99999,-99999,-99999,-99999,-99999,Married,SSC,34,M,10000,246,1.000,1.0,0.990,-99999.0,0,-99999.000,0,0.0,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,-99999,-99999,0,0,53,4,16,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3951,0,0,0,Married,POST-GRADUATE,48,M,15000,75,0.333,0.0,0.000,-99999.0,0,-99999.000,0,0.0,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1


In [18]:
df2.shape

(51336, 62)

### As we can see there are alot of -99999 values in many columns, for that ham ek loop chalayengay k ager kisi column mai 10000 sy zyada -99999 values hai tu osko drop karenagy, our agar oss sy kam hai tu serf oss rows ko drop karengay

In [19]:
columns_to_be_removed = []

for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        columns_to_be_removed.append(i)

In [20]:
columns_to_be_removed

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

### According to our condition, we have to remove all those columns contains in 'columns_to_be_removed' from df2

In [21]:
df2.shape

(51336, 62)

In [22]:
df2 = df2.drop(columns_to_be_removed, axis=1)
print(df2.shape)

(51336, 54)


In [23]:
# Now we will remove all those rows in which the number of -99999 values are less than 10000

In [24]:
df2.shape

(51336, 54)

In [25]:
for i in df2.columns:
    df2 = df2.loc[df2[i] != -99999]

In [26]:
df2.shape

(42066, 54)

In [27]:
df1.isna().sum()

PROSPECTID              0
Total_TL                0
Tot_Closed_TL           0
Tot_Active_TL           0
Total_TL_opened_L6M     0
Tot_TL_closed_L6M       0
pct_tl_open_L6M         0
pct_tl_closed_L6M       0
pct_active_tl           0
pct_closed_tl           0
Total_TL_opened_L12M    0
Tot_TL_closed_L12M      0
pct_tl_open_L12M        0
pct_tl_closed_L12M      0
Tot_Missed_Pmnt         0
Auto_TL                 0
CC_TL                   0
Consumer_TL             0
Gold_TL                 0
Home_TL                 0
PL_TL                   0
Secured_TL              0
Unsecured_TL            0
Other_TL                0
Age_Oldest_TL           0
Age_Newest_TL           0
dtype: int64

In [28]:
print(df2.isna().sum())

PROSPECTID                    0
time_since_recent_payment     0
num_times_delinquent          0
max_recent_level_of_deliq     0
num_deliq_6mts                0
num_deliq_12mts               0
num_deliq_6_12mts             0
num_times_30p_dpd             0
num_times_60p_dpd             0
num_std                       0
num_std_6mts                  0
num_std_12mts                 0
num_sub                       0
num_sub_6mts                  0
num_sub_12mts                 0
num_dbt                       0
num_dbt_6mts                  0
num_dbt_12mts                 0
num_lss                       0
num_lss_6mts                  0
num_lss_12mts                 0
recent_level_of_deliq         0
tot_enq                       0
CC_enq                        0
CC_enq_L6m                    0
CC_enq_L12m                   0
PL_enq                        0
PL_enq_L6m                    0
PL_enq_L12m                   0
time_since_recent_enq         0
enq_L12m                      0
enq_L6m 

## Now we will check the common columns in df1 and df2 so that we can merge them

In [29]:
for i in list(df1.columns):
    if i in list(df2.columns):
        print(i)

PROSPECTID


In [30]:
# Now we will merge df1 and df2 based on PROSPECTID
df = pd.merge(df1, df2, how='inner', left_on=['PROSPECTID'], right_on=['PROSPECTID'])

In [31]:
df.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,Total_TL_opened_L12M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,Auto_TL,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,num_std_6mts,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_6mts,num_dbt_12mts,num_lss,num_lss_6mts,num_lss_12mts,recent_level_of_deliq,tot_enq,CC_enq,CC_enq_L6m,CC_enq_L12m,PL_enq,PL_enq_L6m,PL_enq_L12m,time_since_recent_enq,enq_L12m,enq_L6m,enq_L3m,MARITALSTATUS,EDUCATION,AGE,GENDER,NETMONTHLYINCOME,Time_With_Curr_Empr,pct_of_active_TLs_ever,pct_opened_TLs_L6m_of_L12m,pct_currentBal_all_TL,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,0,0,0.00,0.000,0,0,0,0,1,0,4,1,4,0,72,18,549,11,29,0,0,0,0,0,21,5,11,0,0,0,0,0,0,0,0,0,29,6,0,0,0,6,0,0,566,0,0,0,Married,12TH,48,M,51000,114,0.200,0.0,0.798,0,1,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,1,0,1.00,0.000,0,0,0,1,0,0,0,0,1,0,7,7,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,209,1,0,0,Single,GRADUATE,23,F,19000,50,1.000,0.0,0.370,0,0,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,2,0,0.25,0.000,1,1,0,6,1,0,0,2,6,0,47,2,302,9,25,1,9,8,0,0,10,5,10,0,0,0,0,0,0,0,0,0,25,4,0,0,0,0,0,0,587,0,0,0,Married,SSC,40,M,18,191,1.000,0.5,0.585,0,0,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.0,0.333,0.667,0,0,0.00,0.000,0,1,0,0,0,0,0,3,0,2,131,32,583,0,0,0,0,0,0,0,53,4,16,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3951,0,0,0,Married,POST-GRADUATE,48,M,15000,75,0.333,0.0,0.000,0,0,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.0,0.167,0.833,0,1,0.00,0.167,0,4,0,0,2,0,0,6,0,0,150,17,245,14,270,0,0,0,13,11,5,0,2,3,0,1,0,0,0,0,0,0,26,15,2,0,1,7,3,3,7,6,5,4,Married,12TH,35,M,0,154,0.167,0.0,0.000,0,0,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [33]:
df.isna().sum()

PROSPECTID             0
Total_TL               0
Tot_Closed_TL          0
Tot_Active_TL          0
Total_TL_opened_L6M    0
                      ..
GL_Flag                0
last_prod_enq2         0
first_prod_enq2        0
Credit_Score           0
Approved_Flag          0
Length: 79, dtype: int64

In [34]:
df.shape

(42064, 79)

## We will divide 'Categorical' and 'Numerical' feature and will treat them separately

In [35]:

for i in df.columns:
    if df[i].dtype == 'object':
        print(i)
        


MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


### We will do feature engineering using chi-squared test(for categorical to categorical) on these columns to find out how closely assocaited these features with the target

In [36]:
# Let's perform chi2 test on categorical features to check the association with the target (Features Selection)
for i in ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']:
    chi2, p_value, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, '----', p_value)


MARITALSTATUS ---- 3.578180861038862e-233
EDUCATION ---- 2.6942265249737532e-30
GENDER ---- 1.907936100186563e-05
last_prod_enq2 ---- 0.0
first_prod_enq2 ---- 7.84997610555419e-287


### Since, all the categorical features pval <= 0.05 so we will accept all

### Now we will deal with the numerical features, that how associated them all with the target variable

In [37]:
# Using loop we will extract all the numeric features
numeric_columns = []
for i in df.columns:
    if df[i].dtypes != 'object' and i not in ['PROSPECTID', 'Approved_Flag']:
        numeric_columns.append(i)
    

In [38]:
print(len(numeric_columns))

72


### Since, we have multiple columns therfore will check multicollinearity between all the numeric columns

In [39]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = df[numeric_columns]
columns_to_be_kept = []

for column in numeric_columns:
    vif_value = variance_inflation_factor(vif_data.values, vif_data.columns.get_loc(column))
    print(f'{column} ---- {vif_value}')
    
    if vif_value <= 6:
        columns_to_be_kept.append(column)
    else:
        vif_data = vif_data.drop(column, axis=1)
    
    

c:\Users\FAHIM SABIR\.conda\envs\python_ml\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Total_TL ---- inf


c:\Users\FAHIM SABIR\.conda\envs\python_ml\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Tot_Closed_TL ---- inf
Tot_Active_TL ---- 11.320180023967996
Total_TL_opened_L6M ---- 8.363698035000336
Tot_TL_closed_L6M ---- 6.520647877790928
pct_tl_open_L6M ---- 5.149501618212625
pct_tl_closed_L6M ---- 2.611111040579735


c:\Users\FAHIM SABIR\.conda\envs\python_ml\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


pct_active_tl ---- inf
pct_closed_tl ---- 1788.7926256209232
Total_TL_opened_L12M ---- 8.601028256477228
Tot_TL_closed_L12M ---- 3.8328007921530785
pct_tl_open_L12M ---- 6.099653381646731
pct_tl_closed_L12M ---- 5.5813520096427585
Tot_Missed_Pmnt ---- 1.9855843530987785


c:\Users\FAHIM SABIR\.conda\envs\python_ml\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Auto_TL ---- inf
CC_TL ---- 4.809538302819343
Consumer_TL ---- 23.270628983464636
Gold_TL ---- 30.595522588100053
Home_TL ---- 4.384346405965587
PL_TL ---- 3.064658415523423
Secured_TL ---- 2.898639771299251
Unsecured_TL ---- 4.377876915347322
Other_TL ---- 2.2078535836958433
Age_Oldest_TL ---- 4.916914200506864
Age_Newest_TL ---- 5.214702030064725
time_since_recent_payment ---- 3.3861625024231476
num_times_delinquent ---- 7.840583309478997
max_recent_level_of_deliq ---- 5.255034641721438


c:\Users\FAHIM SABIR\.conda\envs\python_ml\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


num_deliq_6mts ---- inf
num_deliq_12mts ---- 7.380634506427238
num_deliq_6_12mts ---- 1.4210050015175733
num_times_30p_dpd ---- 8.083255010190316
num_times_60p_dpd ---- 1.6241227524040112
num_std ---- 7.257811920140003
num_std_6mts ---- 15.59624383268298
num_std_12mts ---- 1.8258570471324314
num_sub ---- 1.5080839450032664
num_sub_6mts ---- 2.172088834824577
num_sub_12mts ---- 2.623397553527229
num_dbt ---- 2.2959970812106176
num_dbt_6mts ---- 7.360578319196439
num_dbt_12mts ---- 2.160238777310255
num_lss ---- 2.8686288267891467
num_lss_6mts ---- 6.458218003637272
num_lss_12mts ---- 2.8474118865638256
recent_level_of_deliq ---- 4.753198156284083
tot_enq ---- 16.227354755948223
CC_enq ---- 6.424377256363872
CC_enq_L6m ---- 8.887080381808687
CC_enq_L12m ---- 2.3804746142952653
PL_enq ---- 8.609513476514548
PL_enq_L6m ---- 13.06755093547673
PL_enq_L12m ---- 3.500040056654654
time_since_recent_enq ---- 1.908795587481377
enq_L12m ---- 17.006562234161628
enq_L6m ---- 10.730485153719197
enq_L

In [40]:
print(len(columns_to_be_kept))

39


In [41]:
# check Anova for columns_to_be_kept 

from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for column in columns_to_be_kept:
    groups = [df[df['Approved_Flag'] == flag][column].tolist() for flag in ['P1', 'P2', 'P3', 'P4']]
    f_statistic, p_value = f_oneway(*groups)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(column)


In [42]:
print(len(columns_to_be_kept_numerical))

37


In [43]:
# listing all the final features
features = columns_to_be_kept_numerical + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
df = df[features + ['Approved_Flag']]
df.sample(10)

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_12mts,num_lss,recent_level_of_deliq,CC_enq_L12m,PL_enq_L12m,time_since_recent_enq,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2,Approved_Flag
22165,1.000,0.000,0,0.000,0,0,0,0,0,1,0,5,5,70,0,0,0,0,0,0,0,0,0,0,0,0,2,7,2,25000,125,0,0,1.0,0.0,0,0,Married,GRADUATE,M,PL,others,P3
25019,1.000,0.000,0,0.000,0,0,0,0,0,1,0,5,5,87,0,0,0,0,0,0,0,0,0,0,0,0,1,142,0,25000,85,0,0,1.0,0.0,0,0,Married,UNDER GRADUATE,M,PL,others,P3
16863,0.000,0.000,1,0.333,0,0,0,0,0,3,0,18,6,31,27,1,0,0,0,0,0,0,0,0,8,0,0,186,0,34000,66,0,0,0.0,0.0,0,0,Married,GRADUATE,M,ConsumerLoan,ConsumerLoan,P2
35745,0.000,0.500,1,0.500,0,0,0,1,1,1,0,28,27,36,1,0,0,0,0,0,0,0,0,0,1,0,0,0,2,15000,66,0,1,0.0,0.0,0,0,Married,SSC,F,ConsumerLoan,AL,P4
21089,0.111,0.111,1,0.111,2,0,0,0,9,0,8,67,6,120,0,0,0,6,0,0,0,0,0,0,0,0,0,962,0,14000,60,0,0,0.0,0.0,0,0,Married,12TH,M,others,others,P2
27469,0.000,1.000,1,1.000,0,0,0,0,0,1,0,12,12,152,17,1,0,0,0,0,0,0,0,0,17,0,0,372,0,24000,99,0,0,0.0,0.0,0,0,Single,12TH,M,ConsumerLoan,ConsumerLoan,P2
1298,0.000,0.000,1,0.250,0,0,0,0,2,2,2,42,10,336,13,2,11,9,0,0,0,0,0,0,13,0,0,1259,0,20000,47,0,0,0.0,0.0,1,0,Single,SSC,M,others,others,P2
1311,0.000,0.000,0,0.000,0,0,0,0,1,0,0,37,37,345,25,4,0,0,0,0,0,0,0,0,25,0,0,1116,0,23000,66,0,0,0.0,0.0,0,0,Single,SSC,M,others,others,P2
2524,0.000,0.000,0,0.000,0,0,0,0,2,0,0,96,27,1804,0,0,0,0,0,0,0,0,0,0,0,0,0,811,0,20000,221,0,0,0.0,0.0,0,0,Married,12TH,M,AL,AL,P1
30371,0.333,0.000,0,0.000,2,0,0,0,3,3,4,59,3,398,0,0,0,10,0,0,0,0,0,0,0,0,0,88,2,16500,71,0,0,0.0,0.0,1,0,Married,GRADUATE,M,others,others,P2


In [44]:
print(df['MARITALSTATUS'].unique())
print(df['EDUCATION'].unique())
print(df['GENDER'].unique())
print(df['last_prod_enq2'].unique())
print(df['first_prod_enq2'].unique())    

['Married' 'Single']
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
['M' 'F']
['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']



* Ordinal feature -- EDUCATION
* SSC            : 1
* 12TH           : 2
* GRADUATE       : 3
* UNDER GRADUATE : 3
* POST-GRADUATE  : 4
* OTHERS         : 1
* PROFESSIONAL   : 3

In [45]:
# Encoding EDUCATION column giving them separate values
education_mapping = {
    'SSC': 1,
    '12TH': 2,
    'GRADUATE': 3,
    'UNDER GRADUATE': 3,
    'POST-GRADUATE': 4,
    'OTHERS': 1,
    'PROFESSIONAL': 3
}

df['EDUCATION'] = df['EDUCATION'].map(education_mapping)


In [46]:
df['EDUCATION'].value_counts()

EDUCATION
3    18931
2    11703
1     9532
4     1898
Name: count, dtype: int64

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            42064 non-null  float64
 1   pct_tl_closed_L6M          42064 non-null  float64
 2   Tot_TL_closed_L12M         42064 non-null  int64  
 3   pct_tl_closed_L12M         42064 non-null  float64
 4   Tot_Missed_Pmnt            42064 non-null  int64  
 5   CC_TL                      42064 non-null  int64  
 6   Home_TL                    42064 non-null  int64  
 7   PL_TL                      42064 non-null  int64  
 8   Secured_TL                 42064 non-null  int64  
 9   Unsecured_TL               42064 non-null  int64  
 10  Other_TL                   42064 non-null  int64  
 11  Age_Oldest_TL              42064 non-null  int64  
 12  Age_Newest_TL              42064 non-null  int64  
 13  time_since_recent_payment  42064 non-null  int

In [48]:
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'])

# Convert only boolean columns to integers
for col in df_encoded.select_dtypes(include='bool').columns:
    df_encoded[col] = df_encoded[col].astype(int)



In [49]:
df_encoded

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_12mts,num_lss,recent_level_of_deliq,CC_enq_L12m,PL_enq_L12m,time_since_recent_enq,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,EDUCATION,Approved_Flag,MARITALSTATUS_Married,MARITALSTATUS_Single,GENDER_F,GENDER_M,last_prod_enq2_AL,last_prod_enq2_CC,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,0.000,0.00,0,0.000,0,0,0,4,1,4,0,72,18,549,29,0,0,11,0,0,0,0,0,0,29,0,0,566,0,51000,114,0,1,0.000,0.0,1,0,2,P2,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
1,0.000,0.00,0,0.000,0,0,0,0,0,1,0,7,7,47,0,0,0,0,0,0,0,0,0,0,0,0,0,209,0,19000,50,0,0,0.000,0.0,0,0,3,P2,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0
2,0.125,0.00,0,0.000,1,0,0,0,2,6,0,47,2,302,25,8,0,10,0,0,0,0,0,0,25,0,0,587,0,18,191,0,0,0.000,0.0,1,0,1,P2,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1
3,0.000,0.00,0,0.000,0,0,0,0,3,0,2,131,32,583,0,0,0,16,0,0,0,0,0,0,0,0,0,3951,0,15000,75,0,0,0.000,0.0,0,0,4,P1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0
4,0.000,0.00,1,0.167,0,0,0,0,6,0,0,150,17,245,270,0,11,2,3,0,1,0,0,0,26,1,3,7,4,0,154,0,0,0.429,0.0,1,0,2,P3,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,0.333,0.00,0,0.000,0,0,0,0,0,3,1,24,5,15,24,0,0,0,0,0,0,0,0,0,24,0,0,0,1,18500,249,0,0,0.000,0.0,0,0,2,P4,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0
42060,0.000,0.25,1,0.250,0,0,0,0,2,2,0,74,7,57,0,0,0,6,0,0,0,0,0,0,0,0,0,203,0,25000,186,0,0,0.000,0.0,0,0,1,P1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
42061,0.500,0.50,1,0.500,0,0,0,0,0,2,0,9,5,32,0,0,0,0,0,0,0,0,0,0,0,0,2,1,2,18000,66,0,0,1.000,0.0,0,0,1,P3,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1
42062,0.000,0.00,1,0.500,0,0,0,0,0,2,0,15,8,58,0,0,0,0,0,0,0,0,0,0,0,0,0,242,0,12802,54,0,0,0.000,0.0,0,0,3,P2,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1


## Now, we final cleaned well featured DataFrame which is df_encoded, using this we will train our model

In [50]:
x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag']

In [51]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [52]:
# Random Forest
rf_classifier = RandomForestClassifier(n_estimators = 100, random_state=42)
rf_classifier.fit(x_train, y_train)
y_pred = rf_classifier.predict(x_test)

In [53]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy}')
print ()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.7644122191845952

Class p1:
Precision: 0.8341121495327103
Recall: 0.7041420118343196
F1 Score: 0.7636363636363638

Class p2:
Precision: 0.7954429518789321
Recall: 0.9272547076313181
F1 Score: 0.8563060589419733

Class p3:
Precision: 0.4573643410852713
Recall: 0.22264150943396227
F1 Score: 0.29949238578680204

Class p4:
Precision: 0.7216294859359845
Recall: 0.7230320699708455
F1 Score: 0.7223300970873787



In [54]:
# 2. xgboost

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)



y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)




xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy:.2f}')
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.78

Class p1:
Precision: 0.823906083244397
Recall: 0.7613412228796844
F1 Score: 0.7913890312660173

Class p2:
Precision: 0.8255418233924413
Recall: 0.913577799801784
F1 Score: 0.8673315769665036

Class p3:
Precision: 0.4756380510440835
Recall: 0.30943396226415093
F1 Score: 0.3749428440786465

Class p4:
Precision: 0.7342386032977691
Recall: 0.7356656948493683
F1 Score: 0.7349514563106796



In [55]:

# 3. Decision Tree
from sklearn.tree import DecisionTreeClassifier


y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)
dt_model.fit(x_train, y_train)
y_pred = dt_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f"Accuracy: {accuracy:.2f}")
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.71

Class p1:
Precision: 0.7203557312252964
Recall: 0.7189349112426036
F1 Score: 0.71964461994077

Class p2:
Precision: 0.808029623854999
Recall: 0.8218037661050545
F1 Score: 0.8148584905660377

Class p3:
Precision: 0.334916864608076
Recall: 0.3192452830188679
F1 Score: 0.3268933539412674

Class p4:
Precision: 0.6434955312810328
Recall: 0.6297376093294461
F1 Score: 0.6365422396856582



In [56]:
from xgboost import XGBClassifier

y = df_encoded['Approved_Flag']
x = df_encoded.drop(['Approved_Flag'], axis=1)

# Encode target labels if necessary
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

# Initialize classifiers
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_classifier = XGBClassifier(objective='multi:softmax', num_class=4, random_state=42)
dt_classifier = DecisionTreeClassifier(max_depth=20, min_samples_split=10, random_state=42)

# List of classifiers
classifiers = [
    ('Random Forest', rf_classifier),
    ('XGBoost', xgb_classifier),
    ('Decision Tree', dt_classifier)
]

# List to store metrics
metrics_list = []

# Train and evaluate each classifier
for name, clf in classifiers:
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    
    metrics_list.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1_score
    })

# Convert list of metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Print the metrics DataFrame
print(metrics_df)

           Model  Accuracy  Precision    Recall  F1 Score
0  Random Forest  0.764412   0.737830  0.764412  0.741055
1        XGBoost  0.778319   0.759069  0.778319  0.764438
2  Decision Tree  0.711280   0.707346  0.711280  0.709219


In [57]:
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

In [58]:

# Hyperparameter tuning in xgboost
from sklearn.model_selection import GridSearchCV
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

# Define the XGBClassifier with the initial set of hyperparameters
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

# Define the parameter grid for hyperparameter tuning

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model with the best hyperparameters on the test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(x_test, y_test)
print("Test Accuracy:", accuracy)

# Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}

Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}
Test Accuracy: 0.7811719957209081


In [59]:
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

param_grid = {
    'n_estimators': [200],
    'max_depth': [3],
    'learning_rate': [0.2]
}


grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...e,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_class=4,
                                     num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.2], 'max_depth': [3],
                         'n_estimators': [200]},
             scoring='accuracy')

In [60]:
y_pred = grid_search.predict(x_test)

print('The accuracy is', accuracy_score(y_test,y_pred))

The accuracy is 0.7811719957209081


In [62]:
import pickle
pickle.dump(grid_search,open('credit_risk_model.pkl','wb'))